In [1]:
from scapy.all import *
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import seaborn as sns
from sklearn import preprocessing
from datetime import datetime
import plotly.express as px
import statistics

In [2]:
scapy_cap = rdpcap('../Data/rw.pcap')

In [3]:
P_source = []
P_dest = []
P_len = []
P_dt=[]
P_surcePort = []

In [4]:
for packets in scapy_cap:
    P_source.append(packets[IP].src)
    P_dest.append(packets[IP].dst)
    P_len.append(packets[IP].len)
    P_dt.append(packets[IP].time)
    P_surcePort.append(packets[TCP].sport)

In [7]:
P_dt2=[]
for i in range(1,len(P_dt)):
    P_dt2.append(P_dt[i]-P_dt[i-1])


In [9]:
df=pd.DataFrame({'Source':P_source,'Destination':P_dest,'Length':P_len,'DT':P_dt,'Source Port':P_surcePort})

In [10]:
print(df.head())

            Source      Destination  Length            DT  Source Port
0  192.168.137.104    172.217.21.42      60  1.650211e+09        47834
1    172.217.21.42  192.168.137.104      60  1.650211e+09          443
2  192.168.137.104    172.217.21.42      52  1.650211e+09        47834
3  192.168.137.104    172.217.21.42     569  1.650211e+09        47834
4    172.217.21.42  192.168.137.104      52  1.650211e+09          443


In [12]:
df.fillna(0,inplace=True)

In [14]:
myip='192.168.137.104'
dt=df

In [15]:
direction=[]
for i in dt['Destination']:
    if(i==myip):
        direction.append('in')
    else:
        direction.append('out')

In [16]:
dt.insert(0,'direction',direction)

In [20]:
inp=[]
out=[]
leng_mean=[]
leng_std=[]
leng_min=[]
leng_max=[]
sum_dt=[]
port=[]

In [21]:
start=0
end=100
inps=0
outs=0
cc=round(dt.shape[0]/100)
tempdf=[]
tempport=[]
tempdt=0
for i in range(cc):
    for k in range(start,end):
        if(dt['direction'][k]=='in'):
            inps=inps+1
        else:
            outs=outs+1
        tempdf.append(dt['Length'][k])
        tempdt=tempdt+dt['DT'][k]
        tempport.append(dt['Source Port'][k])

        
    leng_mean.append(statistics.mean(tempdf))
    leng_std.append(statistics.stdev(tempdf))
    leng_min.append(min(tempdf))
    leng_max.append(max(tempdf))
    sum_dt.append(tempdt)
    port.append(max(tempport,key=tempport.count))
    
    inp.append(inps)
    out.append(outs)
    start=start+100
    end=end+100
    inps=0
    outs=0
    tempdf=[]
    tempport=[]
    tempdt=0


In [22]:
df=pd.DataFrame()

In [23]:
df.insert(0,'inp',inp)
df.insert(1,'out',out)
df.insert(2,'leng_mean',leng_mean)
df.insert(3,'leng_std',leng_std)
df.insert(4,'leng_min',leng_min)
df.insert(5,'leng_max',leng_max)
df.insert(6,'sum_dt',sum_dt)
df.insert(7,'port',port)

In [24]:
print(df.head())

   inp  out  leng_mean    leng_std  leng_min  leng_max        sum_dt  port
0   56   44        563  610.707786        52      1470  1.650211e+11   443
1   37   63        288  465.745639        52      1470  1.650211e+11   443
2   43   57        391  553.813145        40      1470  1.650211e+11   443
3   42   58        340  498.887763        40      1470  1.650211e+11   443
4   39   61        139  286.623446        52      1470  1.650211e+11   443


In [25]:
import pickle
loaded_model = pickle.load(open('../exe/KNN2.sav', 'rb')) # Load the model 


In [27]:
import joblib
joblib.dump(loaded_model, '../exe/KNN2.sav')